<a href="https://colab.research.google.com/github/RajAakash/TransferLearningPytorch/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
df=pd.read_csv("all_August_2019.csv")
df.head(5)

,runtime,application,algorithm,bw_level,ipath_0,ipath_1,task_count,power_cap,thread_count,year
0,54.4034,CoMD,pak,1.0,2.0,0.0,4096.0,64.0,24.0,2017.0
1,46.5075,CoMD,pak,1.0,2.0,0.0,4096.0,80.0,24.0,2017.0
2,42.4216,CoMD,pak,1.0,2.0,0.0,4096.0,115.0,24.0,2017.0
3,41.7099,CoMD,pak,1.0,2.0,0.0,2048.0,64.0,16.0,2017.0
4,47.5113,CoMD,pak,1.0,2.0,0.0,2048.0,64.0,20.0,2017.0


In [ ]:
df = pd.get_dummies(df, columns=['algorithm', 'application'])
df = df.drop(['ipath_0', 'ipath_1'], axis=1)
df.head(5)

,runtime,bw_level,task_count,power_cap,thread_count,year,algorithm_pak,algorithm_rand,algorithm_spr,application_CoMD,application_amr,application_cg,application_ft,application_kripke,application_lu,application_mg
0,54.4034,1.0,4096.0,64.0,24.0,2017.0,1,0,0,1,0,0,0,0,0,0
1,46.5075,1.0,4096.0,80.0,24.0,2017.0,1,0,0,1,0,0,0,0,0,0
2,42.4216,1.0,4096.0,115.0,24.0,2017.0,1,0,0,1,0,0,0,0,0,0
3,41.7099,1.0,2048.0,64.0,16.0,2017.0,1,0,0,1,0,0,0,0,0,0
4,47.5113,1.0,2048.0,64.0,20.0,2017.0,1,0,0,1,0,0,0,0,0,0


In [ ]:
y = df['runtime']
X = df.drop('runtime', axis=1)
print(X.shape,y.shape)

(3992, 15) (3992,)


In [ ]:
X = X.to_numpy()
y = y.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Load a pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
print(model.parameters)

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [ ]:
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

In [ ]:
summary(model=model,
        input_size=(1,3,224,224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"]
        )

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [1, 3, 224, 224]     [1, 1000]            --                   True
├─Conv2d (conv1)                         [1, 3, 224, 224]     [1, 64, 112, 112]    9,408                True
├─BatchNorm2d (bn1)                      [1, 64, 112, 112]    [1, 64, 112, 112]    128                  True
├─ReLU (relu)                            [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
├─MaxPool2d (maxpool)                    [1, 64, 112, 112]    [1, 64, 56, 56]      --                   --
├─Sequential (layer1)                    [1, 64, 56, 56]      [1, 64, 56, 56]      --                   True
│    └─BasicBlock (0)                    [1, 64, 56, 56]      [1, 64, 56, 56]      --                   True
│    │    └─Conv2d (conv1)               [1, 64, 56, 56]      [1, 64, 56, 56]      36,864               True
│    │    └─BatchN

In [ ]:
# Freeze all layers except the last few
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Define the regression layer
num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_features, 256),  # Add a new hidden layer
    nn.ReLU(),
    nn.Linear(256, 1)  # Output layer for regression (1 output neuron)
)

In [ ]:
summary(model=model,
        input_size=(1,3,224,224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"]
        )

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [1, 3, 224, 224]     [1, 1]               --                   Partial
├─Conv2d (conv1)                         [1, 3, 224, 224]     [1, 64, 112, 112]    (9,408)              False
├─BatchNorm2d (bn1)                      [1, 64, 112, 112]    [1, 64, 112, 112]    (128)                False
├─ReLU (relu)                            [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
├─MaxPool2d (maxpool)                    [1, 64, 112, 112]    [1, 64, 56, 56]      --                   --
├─Sequential (layer1)                    [1, 64, 56, 56]      [1, 64, 56, 56]      --                   False
│    └─BasicBlock (0)                    [1, 64, 56, 56]      [1, 64, 56, 56]      --                   False
│    │    └─Conv2d (conv1)               [1, 64, 56, 56]      [1, 64, 56, 56]      (36,864)             False
│    │    

In [ ]:
#Define Loss and optimizer
criterion=nn.MSELoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    inputs = torch.tensor(X_train, dtype=torch.float32)
    targets = torch.tensor(y_train, dtype=torch.float32)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

In [ ]:
# Evaluate the fine-tuned model
model.eval()
with torch.no_grad():
    inputs = torch.tensor(X_test, dtype=torch.float32)
    targets = torch.tensor(y_test, dtype=torch.float32)
    predictions = model(inputs)

    mse = mean_squared_error(targets, predictions)
    print(f"Mean Squared Error: {mse}")